In [492]:
import tensorflow as tf

import keras

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [410]:
from keras.layers import Input, LSTM, RepeatVector, Dense, Embedding, Activation, Masking,SimpleRNN
from keras.models import Model, Sequential
from keras import backend as K

### Synthetic dataset

- 0 - is a start symbol
- 3 - end symbol
- 4 - empty pos in the end of seq, used to equalize seqences lenghts

Data consists of two type of sequences seq_1 and seq_2

In [457]:
seq_1 = np.ones((100, 20))
seq_1[:, ::2] = 2
seq_1[:, 0] = 0
seq_1[:, -4] = 3
seq_1[:, -3:] = 4
print seq_1[0]

[ 0.  1.  2.  1.  2.  1.  2.  1.  2.  1.  2.  1.  2.  1.  2.  1.  3.  4.
  4.  4.]


In [459]:
seq_2 = np.ones((100, 20))
seq_2[:,0] = 0
seq_2[:,-6] = 3
seq_2[:,-5:] = 4
print seq_2[0]

[ 0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  3.  4.  4.  4.
  4.  4.]


In [460]:
X = np.vstack((seq_1, seq_2))

indexes = np.arange(200)
np.random.shuffle(indexes)
X = X[indexes]

Y is one-hot encoded X

In [470]:
VOC_SIZE = 5
Y = np.zeros((200, 20, VOC_SIZE))

In [471]:
for i in range(200):
    for j in range(20):
        Y[i,j,X.astype(int)[i,j]] = 1

#### Net

In [485]:
encoder_input = Input(shape=(None,))

encoder_mask_layer = Masking(mask_value=3, input_shape=(t_s,))
encoder_mask = encoder_mask_layer(encoder_input)

encoder_embedding_layer = Embedding(VOC_SIZE, out_dim)
encoder_embedding = encoder_embedding_layer(encoder_mask)

encoder_recurent_layer_1 = LSTM(2, return_sequences=True, return_state=True)
encoder_recurent_1, encoder_h, encoder_c = encoder_recurent_layer_1(encoder_embedding)

encoder_out_layer = Dense(VOC_SIZE, activation='softmax')
encoder_out = encoder_out_layer (encoder_recurent_1)

In [486]:
encoder = Model(encoder_input, [encoder_h, encoder_c])

In [502]:
decoder_input = Input(shape=(None,))

decoder_mask_layer = Masking(mask_value=3)
decoder_mask = decoder_mask_layer(decoder_input)

decoder_embedding = encoder_embedding_layer(decoder_mask)

decoder_recurent_layer_1 = LSTM(2,return_sequences=True,return_state=True)
decoder_recurent_1, _, _  = decoder_recurent_layer_1(decoder_embedding, initial_state = [encoder_h, encoder_c])

decoder_out_layer = Dense(VOC_SIZE, activation='softmax')
decoder_out = decoder_out_layer(decoder_recurent_1)

In [503]:
total_model = Model([encoder_input, decoder_input], decoder_out)

##### Learning

In [504]:
total_model.compile(optimizer='adam', loss='categorical_crossentropy')

In [507]:
learning_progress = total_model.fit([X[:,:], X[:,:-1]], Y[:,1:,:],
                                    batch_size=20,
                                    epochs=1000,
                                    validation_split=0.2)

Train on 160 samples, validate on 40 samples
Epoch 1/1000
160/160 [==============================] - 5s 29ms/step - loss: 1.4955 - val_loss: 1.4807
Epoch 2/1000
160/160 [==============================] - 0s 2ms/step - loss: 1.4774 - val_loss: 1.4630
Epoch 3/1000
160/160 [==============================] - 0s 2ms/step - loss: 1.4607 - val_loss: 1.4466

##### Decoder forward model

In [517]:
initial_decoder_h = Input(shape=(2,))
initial_decoder_c = Input(shape=(2,))

decoder_recurent_out, decoder_h, decoder_c = decoder_recurent_layer_1(decoder_embedding,
                                                                      initial_state=[initial_decoder_h, initial_decoder_c])
decoder_true_out  = decoder_out_layer(decoder_recurent_out)

In [518]:
total_decoder = Model([decoder_input, initial_decoder_h, initial_decoder_c], decoder_true_out)

##### Give it a try

In [521]:
X[:2]

array([[ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  3.,  4.,  4.,  4.,  4.,  4.],
       [ 0.,  1.,  2.,  1.,  2.,  1.,  2.,  1.,  2.,  1.,  2.,  1.,  2.,
         1.,  2.,  1.,  3.,  4.,  4.,  4.]])

In [522]:
h_, c_ = encoder.predict(X[:2])

In [523]:
total_decoder.predict([np.array([[0.,  1.,  2.,  1.,  2.]]), np.array([h_[0]]), np.array([c_[0]])])

array([[[  1.01961461e-06,   9.99978542e-01,   8.60564523e-06,
           3.52137413e-08,   1.17457757e-05],
        [  2.25865500e-04,   8.63294423e-01,   1.36432558e-01,
           4.07396437e-05,   6.37956282e-06],
        [  6.09431879e-07,   9.99983668e-01,   3.46102706e-06,
           1.80551130e-08,   1.22618203e-05],
        [  2.91381701e-04,   7.64513373e-01,   2.35131010e-01,
           5.87916074e-05,   5.47482705e-06],
        [  5.67191989e-07,   9.99984145e-01,   3.04781179e-06,
           1.64479985e-08,   1.23359541e-05]]], dtype=float32)